In [23]:
import numpy as np
import pandas as pd
# The statistical module used to run chi square test
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
csv1 = "Data/Behavioral_Risk_2011_clean.csv"

csv2 = "Data/CDC_STATE_System_E-Cigarette_Legislation_-_Tax.csv"

usage_df = pd.read_csv(csv1)
legis_df = pd.read_csv(csv2)
#usage_df.head()
legis_df.head()

,YEAR,Quarter,LocationAbbr,LocationDesc,TopicDesc,MeasureDesc,DataSource,ProvisionGroupDesc,ProvisionDesc,ProvisionValue,...,Comments,Enacted_Date,Effective_Date,GeoLocation,DisplayOrder,TopicTypeId,TopicId,MeasureId,ProvisionGroupID,ProvisionID
0,2016,2,VI,U.S. Virgin Islands,Legislation - E-Cigarette - Tax,E-Cigarette,OSH,Restrictions,Percent Value,45,...,NaN,3/23/2016,NaN,"(18.335765, -64.896335)",3,LEG,1040LEG,1001ETR,10GRP,452
1,2019,4,OH,Ohio,Legislation - E-Cigarette - Tax,E-Cigarette,OSH,Restrictions,E-Cigarette Tax,Yes,...,NaN,7/18/2019,10/17/2019,"(40.06021014100048, -82.40426005599966)",1,LEG,1040LEG,1001ETR,10GRP,450
2,2016,3,VI,U.S. Virgin Islands,Legislation - E-Cigarette - Tax,E-Cigarette,OSH,Restrictions,Percent Value,45,...,NaN,3/23/2016,NaN,"(18.335765, -64.896335)",3,LEG,1040LEG,1001ETR,10GRP,452
3,2019,4,OH,Ohio,Legislation - E-Cigarette - Tax,E-Cigarette,OSH,Restrictions,E-Cigarette Tax ($ each),0.01,...,NaN,7/18/2019,10/17/2019,"(40.06021014100048, -82.40426005599966)",2,LEG,1040LEG,1001ETR,10GRP,451
4,2019,4,OH,Ohio,Legislation - E-Cigarette - Tax,E-Cigarette,OSH,Restrictions,Type of Tax,Per vapor volume,...,NaN,7/18/2019,10/17/2019,"(40.06021014100048, -82.40426005599966)",4,LEG,1040LEG,1001ETR,10GRP,453


In [24]:
legis_drop_df = legis_df[legis_df.ProvisionValue !="No Provision"]
legis_count_df = legis_drop_df.groupby(['LocationAbbr']).agg({'ProvisionGroupDesc':'count'})
legis_count_df.head()

,ProvisionGroupDesc
LocationAbbr,
CA,42
CT,12
DC,60
DE,33
IL,15


In [25]:
usage_clean_df = usage_df

usage_clean_df = usage_clean_df[usage_clean_df.MeasureDesc !="Frequency of Use"]
usage_clean_df = usage_clean_df[usage_clean_df.Gender !="Overall"]
usage_clean_df = usage_clean_df[usage_clean_df.Response !="Not Current"]
usage_clean_df = usage_clean_df[usage_clean_df.Response !="Former"]
usage_clean_df = usage_clean_df[usage_clean_df.Response !="Never"]

usage_clean_df = usage_clean_df[usage_clean_df.TopicDesc !="Cigarette Use_Adults"]
usage_clean_df = usage_clean_df[usage_clean_df.TopicDesc !="Smokeless Tobacco Use (Adults)"]
usage_clean_df = usage_clean_df[usage_clean_df.TopicDesc !="Cessation (Adults)"]

usagebystate_df = usage_clean_df.groupby(['LocationAbbr']).agg({'Data_Value':['mean']})
usagebystate_df.head()

,Data_Value
,mean
LocationAbbr,
AK,16.238182
AL,18.632000
AR,20.080000
AZ,13.526000
CA,9.780000


In [28]:
combo_df = legis_count_df.merge(usagebystate_df, left_on='LocationAbbr', right_on='LocationAbbr')

combo_df.sort_values(by='ProvisionGroupDesc', ascending=False)

,ProvisionGroupDesc,"(Data_Value, mean)"
LocationAbbr,,
MN,123,13.434545
NC,66,15.260000
LA,63,18.501818
DC,60,13.490000
WV,51,23.570000
PA,51,16.966000
KS,45,16.046000
CA,42,9.780000
DE,33,14.847273


In [6]:
usagebystate_df["Data_Value"].mean()

mean    15.293602
dtype: float64

In [7]:
usagebystate_df.count()

Data_Value  mean    53
dtype: int64

In [8]:
usagebystate_df[1]=15.293602

In [10]:
usagebystate_df.columns = ["observed", "expected"]
usagebystate_df.head()

,observed,expected
LocationAbbr,,
AK,16.238182,15.293602
AL,18.632000,15.293602
AR,20.080000,15.293602
AZ,13.526000,15.293602
CA,9.780000,15.293602


In [1]:
# Calculate the critical value
critical_value = stats.chi2.ppf(q = 0.95, df = 52)
critical_value

NameError: name 'stats' is not defined

In [11]:
# Run the chi square test with stats.chisquare()
stats.chisquare(usagebystate_df['observed'], usagebystate_df['expected'])

Power_divergenceResult(statistic=32.84537048494395, pvalue=0.9825063033993281)